In [18]:
##libraries required
import math
import re
from nltk.metrics import ConfusionMatrix
import random
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy
import csv
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import pandas as pd
import seaborn
from nltk.metrics import ConfusionMatrix
import nltk
from nltk.stem import WordNetLemmatizer 
  

In [2]:
#https://www.kaggle.com/sebastienverpile/consumercomplaintsdata

In [26]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritesh.ratti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ritesh.ratti/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# READ JOINED DATA

In [4]:
pdfJoinedFinal = pd.read_csv('joineddata.csv')


In [5]:
pdfJoinedFinal.head()

,COMPLAINT_ID,COMPLAINT_TEXT,WAS_USER_DISPUTED,DATE_COMPLAINT_USER,PRODUCT_ID,ISSUE_ID,COMPANY,COMPANY_RESPONSE_TO_USER,COMPANY_RESPONSE_TO_PUBLIC,WAS_RESPONSE_TIMELY,DATE_COMPLAINT_COMPANY,MAIN_ISSUE,SUB_ISSUE,MAIN_PRODUCT,SUB_PRODUCT
0,3184195,XXXX and Transunion are reporting incorrectly...,No,03/19/2019,26,253,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with explanation,Company has responded to the consumer and the ...,Yes,03/19/2019,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"Credit reporting, credit repair services, or o...",Credit reporting
1,3184692,XXXX and Transunion are reporting incorrectly ...,No,03/19/2019,26,253,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with explanation,Company has responded to the consumer and the ...,Yes,03/19/2019,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"Credit reporting, credit repair services, or o...",Credit reporting
2,3182430,"3 company with inconsistencies, violations and...",No,03/17/2019,26,253,"McCarthy, Burgess & Wolff, Inc.",Closed with explanation,Company believes it acted appropriately as aut...,Yes,03/17/2019,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"Credit reporting, credit repair services, or o...",Credit reporting
3,3181575,"XXXX, XXXX, and Transunion are not taking my c...",No,03/16/2019,26,253,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with explanation,Company has responded to the consumer and the ...,Yes,03/16/2019,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"Credit reporting, credit repair services, or o...",Credit reporting
4,3181191,"Transunion, XXXX, and XXXX Dispute with all th...",No,03/15/2019,26,253,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed with explanation,Company has responded to the consumer and the ...,Yes,03/15/2019,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,"Credit reporting, credit repair services, or o...",Credit reporting


In [9]:
pdfJoinedFinal['COMPLAINT_TEXT'].values

array(['XXXX  and Transunion are reporting incorrectly that I am 120 days past due on loans with the XXXX  - partial account numbers XXXX ; XXXX ; XXXX ; XXXX.. These accounts reflect a {$0.00} balance and a {$0.00} past due. I have contacted the two bureaus and requested these coding errors be corrected. This incorrect reporting is harming my credit score. It is  impossible to be past due on accounts that have a {$0.00} balance and {$0.00} past due.',
       'XXXX and Transunion are reporting incorrectly that I am 120 days past due on a loan with the XXXX - partial account numberXXXX. This account reflects a {$0.00} balance and a {$0.00} past due. I have contacted the two bureaus and requested these coding errors be corrected. This incorrect reporting is harming my credit score. It is impossible to be past due on accounts that have a {$0.00} balance and {$0.00} past due.',
       '3 company with inconsistencies, violations and double jeopardy ... \nI requested to XXXX  an investigatio

In [7]:
pdfJoinedFinal.columns

Index(['COMPLAINT_ID', 'COMPLAINT_TEXT', 'WAS_USER_DISPUTED',
       'DATE_COMPLAINT_USER', 'PRODUCT_ID', 'ISSUE_ID', 'COMPANY',
       'COMPANY_RESPONSE_TO_USER', 'COMPANY_RESPONSE_TO_PUBLIC',
       'WAS_RESPONSE_TIMELY', 'DATE_COMPLAINT_COMPANY', 'MAIN_ISSUE',
       'SUB_ISSUE', 'MAIN_PRODUCT', 'SUB_PRODUCT'],
      dtype='object')

In [10]:
set(pdfJoinedFinal['MAIN_PRODUCT'].values)

{'Bank account or service',
 'Checking or savings account',
 'Consumer Loan',
 'Credit card',
 'Credit card or prepaid card',
 'Credit reporting',
 'Credit reporting, credit repair services, or other personal consumer reports',
 'Debt collection',
 'Money transfer, virtual currency, or money service',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Payday loan, title loan, or personal loan',
 'Prepaid card',
 'Student loan',
 'Vehicle loan or lease',
 'Virtual currency'}

In [11]:
pdfJoinedFinal.count()

COMPLAINT_ID                  383066
COMPLAINT_TEXT                383066
WAS_USER_DISPUTED             383066
DATE_COMPLAINT_USER           383066
PRODUCT_ID                    383066
ISSUE_ID                      383066
COMPANY                       383066
COMPANY_RESPONSE_TO_USER      383066
COMPANY_RESPONSE_TO_PUBLIC    383066
WAS_RESPONSE_TIMELY           383066
DATE_COMPLAINT_COMPANY        383066
MAIN_ISSUE                    383066
SUB_ISSUE                     383066
MAIN_PRODUCT                  383066
SUB_PRODUCT                   330893
dtype: int64

In [12]:
pdfJoinedFinalDropped = pdfJoinedFinal.dropna().drop_duplicates()

# Data Pre Processing

In [19]:

def readDataPandas(df): 

    print("length of data frame:" + str(len(df)))
    dataX=[]
    dataY=[]
    for i in range(0,len(df)):
        dataY.append(df.MAIN_PRODUCT[i])
        dataX.append(getfeaturesPandas(df,i))
        if i!=0 and i%10000 == 0:
            print('Read 10000 records')
    return dataX,dataY

In [20]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%m/%d/%Y")
    d2 = datetime.strptime(d2, "%m/%d/%Y")
    return abs((d2 - d1).days)

# Tokenization Lemmatization and stop Words Removal
lemmatizer = WordNetLemmatizer() 
def getNormalizedSentence(sentence):
    # Tokenize
    words = nltk.word_tokenize(sentence)
    newwords = []
    #Lemmatize
    for word in words:
        newwords.append(lemmatizer.lemmatize(word))
        
    # Remove STop Words
    stop_words = set(stopwords.words('english'))
    afterStopWordsRemoval =[]
    for w in newwords:
        if w not in stop_words:
            afterStopWordsRemoval.append(w)
            
            
    return ' '.join(afterStopWordsRemoval)

# FEATURES

In [21]:
def getfeaturesPandas(df,i):
  
    featureVals = featuresPandas(df,i)
    featureArray=[]
    featureArray.append(featureVals['COMPLAINT_TEXT'])
    featureArray.append(featureVals['WAS_USER_DISPUTED'])
    featureArray.append(featureVals['DATE_COMPLAINT_USER'])
    featureArray.append(featureVals['COMPANY'])
    featureArray.append(featureVals['COMPANY_RESPONSE_TO_USER'])
    featureArray.append(featureVals['COMPANY_RESPONSE_TO_PUBLIC'])
    featureArray.append(featureVals['WAS_RESPONSE_TIMELY'])
    featureArray.append(featureVals['DATE_COMPLAINT_COMPANY'])
    featureArray.append(featureVals['MAIN_ISSUE'])
    featureArray.append(featureVals['SUB_ISSUE'])
    featureArray.append(featureVals['DIFF_DATE'])
    featureArray.append(featureVals['YEAR'])
    featureArray.append(featureVals['MONTH'])
    featureArray.append(featureVals['DAY'])
    
                    
    return featureArray

In [22]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [23]:
def featuresPandas(df,i): #feature encoder
    return {

        'COMPLAINT_TEXT' : getNormalizedSentence(df.COMPLAINT_TEXT[i]),
        'WAS_USER_DISPUTED' : 1 if df.WAS_USER_DISPUTED[i] == 'Yes' else 0,
        'DATE_COMPLAINT_USER' : df.DATE_COMPLAINT_USER[i],
        'COMPANY' : df.COMPANY[i],
        'COMPANY_RESPONSE_TO_USER' : getNormalizedSentence(df.COMPANY_RESPONSE_TO_USER[i]),
        'COMPANY_RESPONSE_TO_PUBLIC' : getNormalizedSentence(df.COMPANY_RESPONSE_TO_PUBLIC[i]),
        'WAS_RESPONSE_TIMELY' : 1 if df.WAS_RESPONSE_TIMELY[i] == 'Yes' else 0,
        'DATE_COMPLAINT_COMPANY' : df.DATE_COMPLAINT_COMPANY[i],
        'MAIN_ISSUE' : getNormalizedSentence(df.MAIN_ISSUE[i]),
        'SUB_ISSUE' : getNormalizedSentence(df.SUB_ISSUE[i]), 
        'DIFF_DATE' : days_between (df.DATE_COMPLAINT_USER[i],df.DATE_COMPLAINT_COMPANY[i]),
        'YEAR' :  datetime.strptime(df.DATE_COMPLAINT_USER[i], "%m/%d/%Y").year , 
        'MONTH' : datetime.strptime(df.DATE_COMPLAINT_USER[i], "%m/%d/%Y").month ,
        'DAY': datetime.strptime(df.DATE_COMPLAINT_USER[i], "%m/%d/%Y").day 
            }

In [24]:
# READ DATA
df = pd.read_csv('joineddata.csv')


In [25]:
df = df[:1000]

In [26]:

dataX, dataY=readDataPandas(df)
n_samples=len(dataX)


length of data frame:1000


In [29]:
len(dataX)

1000

In [30]:
dataX[0]

['XXXX Transunion reporting incorrectly I 120 day past due loan XXXX - partial account number XXXX ; XXXX ; XXXX ; XXXX .. These account reflect { $ 0.00 } balance { $ 0.00 } past due . I contacted two bureau requested coding error corrected . This incorrect reporting harming credit score . It impossible past due account { $ 0.00 } balance { $ 0.00 } past due .',
 0,
 '03/19/2019',
 'TRANSUNION INTERMEDIATE HOLDINGS, INC.',
 'Closed explanation',
 'Company ha responded consumer CFPB chooses provide public response',
 1,
 '03/19/2019',
 "Problem credit reporting company 's investigation existing problem",
 'Their investigation fix error report',
 0,
 2019,
 3,
 19]

In [31]:


cols = [ 'COMPLAINT_TEXT' , 'WAS_USER_DISPUTED' , 'DATE_COMPLAINT_USER' , 'COMPANY' ,'COMPANY_RESPONSE_TO_USER' , 'COMPANY_RESPONSE_TO_PUBLIC', 'WAS_RESPONSE_TIMELY' ,'DATE_COMPLAINT_COMPANY' , 'MAIN_ISSUE' , 'SUB_ISSUE' ,'DIFF_DATE' ,'YEAR' , 'MONTH' , 'DAY']

In [32]:
len(cols)

14

In [33]:
pdfProcessed = pd.DataFrame(dataX , columns=cols)

In [37]:
pdfProcessed.head(2)

,COMPLAINT_TEXT,WAS_USER_DISPUTED,DATE_COMPLAINT_USER,COMPANY,COMPANY_RESPONSE_TO_USER,COMPANY_RESPONSE_TO_PUBLIC,WAS_RESPONSE_TIMELY,DATE_COMPLAINT_COMPANY,MAIN_ISSUE,SUB_ISSUE,DIFF_DATE,YEAR,MONTH,DAY
0,XXXX Transunion reporting incorrectly I 120 da...,0,03/19/2019,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed explanation,Company ha responded consumer CFPB chooses pro...,1,03/19/2019,Problem credit reporting company 's investigat...,Their investigation fix error report,0,2019,3,19
1,XXXX Transunion reporting incorrectly I 120 da...,0,03/19/2019,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Closed explanation,Company ha responded consumer CFPB chooses pro...,1,03/19/2019,Problem credit reporting company 's investigat...,Their investigation fix error report,0,2019,3,19


In [38]:
pdfProcessed.to_csv('pdf_processed_data.csv')

In [ ]:
# Read Data 

In [39]:
pdfProcessed = pd.read_csv('pdf_processed_data.csv')

In [34]:
pdfProcessed  = pdfProcessed.fillna('')

In [35]:
pdfProcessed.count()

COMPLAINT_TEXT                1000
WAS_USER_DISPUTED             1000
DATE_COMPLAINT_USER           1000
COMPANY                       1000
COMPANY_RESPONSE_TO_USER      1000
COMPANY_RESPONSE_TO_PUBLIC    1000
WAS_RESPONSE_TIMELY           1000
DATE_COMPLAINT_COMPANY        1000
MAIN_ISSUE                    1000
SUB_ISSUE                     1000
DIFF_DATE                     1000
YEAR                          1000
MONTH                         1000
DAY                           1000
dtype: int64

# TFIDF Vectorize the COMPLAINT_TEXT

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer( min_df=5, norm='l2', stop_words='english', max_features=1000, lowercase=True)
complaintfeatures = v.fit_transform(pdfProcessed['COMPLAINT_TEXT']).toarray()



In [37]:
pdfProcessed['COMPLAINT_TEXT'][0]

'XXXX Transunion reporting incorrectly I 120 day past due loan XXXX - partial account number XXXX ; XXXX ; XXXX ; XXXX .. These account reflect { $ 0.00 } balance { $ 0.00 } past due . I contacted two bureau requested coding error corrected . This incorrect reporting harming credit score . It impossible past due account { $ 0.00 } balance { $ 0.00 } past due .'

In [44]:
# Dave tfidf vectorizer
with open('tfidfvectorizer.pkl', 'wb') as fid:
    pickle.dump(v, fid)


In [45]:
complaintfeatures.shape

(383066, 1000)

In [46]:
complaintfeaturesPd = pd.DataFrame(complaintfeatures)

In [47]:
complaintfeaturesPd.head(10)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.337695,0.0,0.0,0.0,0.0,0.209146,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.276358,0.0,0.000000,0.0,0.0
1,0.360271,0.0,0.0,0.0,0.0,0.223129,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.098278,0.0,0.000000,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.385496,0.0,0.000000,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.142196,0.0,0.000000,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.464054,0.0,0.000000,0.0,0.0
5,0.383360,0.0,0.0,0.0,0.0,0.474857,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.104576,0.0,0.000000,0.0,0.0
6,0.166653,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.425157,0.530378,0.0,0.032655,0.0,0.0
7,0.395517,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.215785,0.0,0.000000,0.0,0.0
8,0.000000,0.0,0.0,0.0,0.0,0.000000,0.210917,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.120016,0.0,0.000000,0.0,0.0
9,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.195867,0.111699,0.0,0.000000,0.0,0.0


In [ ]:
# Final Data Frame

In [48]:
finalPd = pd.concat([pdfProcessed,complaintfeaturesPd], axis=1)

In [49]:
cols= finalPd.columns
finalCols = cols.delete([0,1])
finalPdWrite = finalPd[finalCols]
# Save final features
finalPdWrite.to_csv('finalfeatures.csv')